In [18]:
import torch
from torch import tensor
import torch.nn as nn
import torch.optim as optim
from torchvision.transforms import functional as F
import torch.backends.cudnn as cudnn

import torchvision
from torchvision import datasets, transforms
from torchvision.models import resnet18, resnet34
import os
import numpy as np

In [19]:
! nvidia-smi

Thu Jul 25 14:20:18 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 522.06       Driver Version: 522.06       CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   40C    P8     7W /  N/A |   1980MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [20]:

#importing RetinaNet
from torchvision.models.detection import retinanet_resnet50_fpn, RetinaNet_ResNet50_FPN_Weights

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = retinanet_resnet50_fpn(weights=RetinaNet_ResNet50_FPN_Weights.DEFAULT)
model.eval()
model.to(device)

RetinaNet(
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(inplace=True)
          (downsample): Sequential(
            (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): FrozenBatchNorm2d(256, eps=0.0)


In [21]:
#load image
from PIL import Image


def load_image(image_path):
    image = Image.open(image_path).convert("RGB")
    image = F.to_tensor(image).unsqueeze(0)  # Convert to tensor and add batch dimension
    return image

image_path = "images/2.jpg"  # Replace with your image path
image = load_image(image_path).to(device)

In [22]:
# object detection

with torch.no_grad():
    outputs = model(image)

threshold = 0.5
outputs = [{k: v.to('cpu').numpy() for k, v in t.items()} for t in outputs]

boxes = outputs[0]['boxes']
scores = outputs[0]['scores']
labels = outputs[0]['labels']

filtered_indices = np.where(scores > threshold)[0]
filtered_boxes = boxes[filtered_indices]
filtered_scores = scores[filtered_indices]
filtered_labels = labels[filtered_indices]

RuntimeError: Couldn't load custom C++ ops. This can happen if your PyTorch and torchvision versions are incompatible, or if you had errors while compiling torchvision from source. For further information on the compatible versions, check https://github.com/pytorch/vision#installation for the compatibility matrix. Please check your PyTorch version with torch.__version__ and your torchvision version with torchvision.__version__ and verify if they are compatible, and if not please reinstall torchvision so that it matches your PyTorch install.

In [ ]:
# # execution pipline for 1 image
#
# from PIL import Image
# import matplotlib.pyplot as plt
# from matplotlib.patches import Rectangle
#
# img = Image.open('images/2.jpg')
# img_tensor = torchvision.transforms.functional.to_tensor(img)
# img_tensor = img_tensor.to(device)
# # img.show() show image with default app on windows/linux
#
# model.eval()
# with torch.no_grad():
#     list_img_tensor = [img_tensor]
#     obj_det_list = model(list_img_tensor)
#     print(obj_det_list)
#
#     boxes = obj_det_list[0]['boxes']
#     scores = obj_det_list[0]['scores']
#     labels = obj_det_list[0]['labels']
#
#     print(scores)
#
#     np_array = img_tensor.permute(1, 2, 0).to('cpu').numpy()
#     plt.imshow(np_array)
#     ax = plt.gca()
#     for box, score, label in zip(boxes, scores, labels):
#         if score > 0.2:
#             rect = Rectangle((box[0], box[1]),
#                              (box[2] - box[0]),
#                              (box[3] - box[1]),
#                              fill=False,
#                              edgecolor=(0, 0, 1),
#                              linewidth=2)
#             ax.add_patch(rect)
#
# plt.show()